## Forward contracts 

### Why we need them

Imagine you own a steel company and you are responsible for its financial prosperity. You just signed a deal for next year and under normal circumstances you will not face solvency problems. However, your friend informed you that inflation might go up next year and there is chance of a war that could have a significant impact on oil prices. You've got a hunch that your company's energy bills will skyrocket next year. If only there was a way to hedge against this... and that's why we have forward contracts: you agree with the electricity producer how much electricity you would like to buy from them next year for a price you agree on now - before there is a turmoil in the market.


P.S.: The steel you agreed to sell - chances are that you don't get your money straight away either and you agree on another forward contract with the buyer.

### Definition 

Forward contract is an over-the-counter financial derivative whereby a seller agrees to sell the underlying to the buyer at a fixed priced at a defined future date.

Over-the-counter means the contract is not sold on an exchange and is custom designed between the two parties. This implies additional credit risk in case one party fails to deliver on their obligation (e.g. you have cash flow problems or the electricity producer cannot provide sufficient electricity output).

When a buyer and seller step into a forward contract, there is no money exchange. The price of the contract reflects the price for which the buyer agrees to purchase the asset at an expiration date T.

### Price of forward contracts

Forward contract price assuming no costs and benefits arising from holding the asset:
$$ F_{0}(T) = S_{0}(1+r)^T $$
where:
<ul>
    <li>
    T is the expiration date of the contract. This is when the money transfer and change of ownership of the asset happen.
    </li>
<li>
    r is the risk-free interest rate. This is often the rate provided by a US government bond as the government of a developed country is unlikely to default on their debt.
</li>
<li>
    $S_{0}$ is the asset price when the two parties agree on the contract.
</li>
</ul>

There are often benefits and costs associated with holding an asset. The benefit of holding a share is its dividend payment. The benefits don't have to be monetary - the ability to live in a nice house is a perk of owning it. The costs on the other hand can include transactions costs or utility bills and property tax.

With the costs and benefits considered, the price of a forward contract becomes:
$$ F_{0}(T) = (S_{0} - \gamma + \theta)(1+r)^T $$ 

where:
<ul>
    <li>
        $\gamma$ represents the benefits 
    </li>
    <li>
        $\theta$ represents the costs 
    </li>
</ul>

<b>Note that</b> the price of the forward contract subtracts the benefits and adds the costs. This is because the benefits come out of holding the asset, and thus the buyer in the forward contract will not receive them, decreasing its value. Inversely, the buyer in the forward doesn't have to pay the costs of holding the asset, increasing the value of the contract.



Below is a chart that shows how the price of the forward contract changes depending on its expiry rate. Initial asset price is $100. You can select interest by interacting with the slider.

In [104]:
# initial asset price - $100
asset_price = 100

# interest from 0% to 10% with a step of 0.25%
interest_rates = list(np.arange(0, 10, 0.25))

# forward contract expiring in 5 years
time_unit = 'y'
num_units = 20
time = list(range(num_units+1))

build_forward_contract_chart(dates, asset_price, interest_rates, time_unit)

### Value of forward contracts

Value of the forward contract at expiration for the long party:
$$ V_{t}(T) = S_{T} - F_{0}(T) $$

In [70]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [77]:
'''
Math functions
'''
def compute_forward_price(S0, r, T, gamma=0, theta=0, date_type='y'):
    '''
    Computes forward contract price for an asset with spot price S0 with the contract 
    expiring in time T. Risk-free interest rate is r
    '''
    # change interest rate from percentage to numeric
    r = r/100
    
    # express T in years
    if date_type == 'm':
        T = T/12        
    elif date_type == 'd':
        T = T/365
    
    return (S0 - gamma + theta) * (1 + r) ** T


def compute_expiry_forward_value(ST, F0):
    '''
    Computes value of the forward contract at expiry date T
    for long party
    '''
    return ST - F0


def compute_forward_value(St, F0, r, t, T, gamma=0, theta=0):
    '''
    Computes the value of a forward contract at time t for an asset with spot price St
    The contract expires at time T (i.e. T-t time remains until expiry); the initial price 
    of the forward contract (without costs & benefits) considered was F0; the risk-free
    interest rate is r
    '''
    discounted_F = F0 / (1+r) ^ (T-t)
    benefit_cost_value = (gamma - theta) * ((1 + r) ^ t)
    
    return St - benefit_cost_value - discounted_F

In [103]:
'''
Utils
'''
# from: https://stackoverflow.com/questions/11443737/how-to-generate-exponentially-increasing-range-in-python
def generate_asset_prices(order=4):
    numbers_sizes = [i*10**exp for exp in range(1, order) for i in range(1, 10)]
    return numbers_sizes


def get_forward_prices(asset_prices, int_rates, dates, date_type='y'):
    forward_prices = []
    for T in dates:
        forward_prices_at_T = get_forward_prices_at_time(asset_prices, int_rates, T, date_type)
        forward_prices += forward_prices_at_T
        
    return forward_prices
        
        
def get_forward_prices_at_time(asset_prices, int_rates, T, date_type='y'):
    forward_prices = []
    for S0 in asset_prices:
        forward_prices += [compute_forward_price(S0, r, T, date_type=date_type) for r in int_rates]
    
    return forward_prices


def get_ids(asset_prices, int_rates, days):
    ids = []
    for day in days:
        ids += [day] * len(int_rates) * len(asset_prices)
    return ids


def get_asset_prices(asset_prices, int_rates, days):
    asset_prices_l = []
    for day in days:
        for asset_p in asset_prices:
            asset_prices_l += [asset_p] * len(int_rates)
    return asset_prices_l


def get_interest_rates(asset_prices, int_rates, days):
    return int_rates * len(asset_prices) * len(days)


def get_days(asset_prices, int_rates, days):
    days_l = []
    for day in days:
        days_l += [day] * len(asset_prices) * len(int_rates)
        
    return days_l


def build_forward_contract_df(dates, asset_price, interest_rates, date_type='m'):
    '''
    Builds a dataframe that contains details of forward pricing
    DF columns:
        time
        forward_price
        asset_price
        interest_rate
    '''
    asset_prices = [asset_price]
    
    ids = get_ids(asset_prices, interest_rates, dates)
    forward_prices_l = get_forward_prices(asset_prices, interest_rates, dates, date_type)
    interest_rates_l = get_interest_rates(asset_prices, interest_rates, dates)
    asset_prices_l = get_asset_prices(asset_prices, interest_rates, dates)
    dates_l = get_days(asset_prices, interest_rates, dates)

    d = {'id': ids, 
         'forward_price': forward_prices_l, 
         'time': dates_l, 
         'asset_price': asset_prices_l,
         'interest_rate': interest_rates_l
        }

    forward_df = pd.DataFrame(data=d)
    
    return forward_df


def build_forward_contract_chart(dates, asset_price, interest_rates, date_type='m'):
    forward_df = build_forward_contract_df(dates, asset_price, interest_rates, date_type)
    fig = px.scatter(forward_df, x="time", y="forward_price", animation_frame="interest_rate",
                    title='Price of forward contract for an asset with starting price of $100.',
                     labels={
                         "time": f"Expiry date ({date_type})",
                         "forward_price": "Forward price ($)",
                         "interest_rate": "Interest rate (%)"
                     },
                    range_x=[forward_df.time.min(), contract_df.time.max()+1], 
                    range_y=[forward_df.forward_price.min()-50,forward_df.forward_price.max()+25])

    fig["layout"].pop("updatemenus") # optional, drop animation buttons
    fig.show()